In [1]:
import pandas as pd

In [2]:
yearly_data = pd.read_csv('data/london_yearly_housing.csv')
monthly_data = pd.read_csv('data/london_monthly_housing.csv')

In [3]:
yearly_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code               1071 non-null   object 
 1   area               1071 non-null   object 
 2   date               1071 non-null   object 
 3   median_salary      1049 non-null   float64
 4   life_satisfaction  352 non-null    float64
 5   mean_salary        1071 non-null   object 
 6   recycling_pct      860 non-null    object 
 7   population_size    1018 non-null   float64
 8   number_of_jobs     931 non-null    float64
 9   area_size          666 non-null    float64
 10  no_of_houses       666 non-null    float64
 11  borough_flag       1071 non-null   int64  
dtypes: float64(6), int64(1), object(5)
memory usage: 100.5+ KB


In [4]:
monthly_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13549 entries, 0 to 13548
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           13549 non-null  object 
 1   area           13549 non-null  object 
 2   average_price  13549 non-null  int64  
 3   code           13549 non-null  object 
 4   houses_sold    13455 non-null  float64
 5   no_of_crimes   7439 non-null   float64
 6   borough_flag   13549 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 741.1+ KB


## Preprocessing

### 1) Convert date column type to datetime

In [5]:
yearly_data['date'] = pd.to_datetime(yearly_data['date'])
monthly_data['date'] = pd.to_datetime(monthly_data['date'])

In [6]:
yearly_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1071 entries, 0 to 1070
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   code               1071 non-null   object        
 1   area               1071 non-null   object        
 2   date               1071 non-null   datetime64[ns]
 3   median_salary      1049 non-null   float64       
 4   life_satisfaction  352 non-null    float64       
 5   mean_salary        1071 non-null   object        
 6   recycling_pct      860 non-null    object        
 7   population_size    1018 non-null   float64       
 8   number_of_jobs     931 non-null    float64       
 9   area_size          666 non-null    float64       
 10  no_of_houses       666 non-null    float64       
 11  borough_flag       1071 non-null   int64         
dtypes: datetime64[ns](1), float64(6), int64(1), object(4)
memory usage: 100.5+ KB


### 2) For simplicity, use only some features and drop the rest

In [8]:
relevant_yearly_features = ['area', 'date', 'median_salary', 'population_size']
relevant_monthly_features = ['area', 'date', 'average_price']

In [9]:
yearly_data = yearly_data[relevant_yearly_features]
monthly_data = monthly_data[relevant_monthly_features]

In [11]:
yearly_data = yearly_data.dropna(subset=['median_salary', 'population_size'])

In [12]:
print(len(yearly_data))
print(len(monthly_data))

999
13549


### 3) Get the year from date

In [13]:
yearly_data['year'] = pd.DatetimeIndex(yearly_data['date']).year
monthly_data['year'] = pd.DatetimeIndex(monthly_data['date']).year

monthly_data = monthly_data.drop(['date'], axis=1)

### 4) Take the mean of monthly data and merge with annual data

In [14]:
mean_from_monthly = monthly_data.groupby(['area', 'year']).mean().reset_index()
mean_from_monthly

,area,year,average_price
0,barking and dagenham,1995,51818.000000
1,barking and dagenham,1996,51718.250000
2,barking and dagenham,1997,55974.250000
3,barking and dagenham,1998,60285.750000
4,barking and dagenham,1999,65320.833333
...,...,...,...
1165,yorks and the humber,2016,149015.583333
1166,yorks and the humber,2017,154819.916667
1167,yorks and the humber,2018,159936.416667
1168,yorks and the humber,2019,163584.583333


In [15]:
mean_from_monthly

,area,year,average_price
0,barking and dagenham,1995,51818.000000
1,barking and dagenham,1996,51718.250000
2,barking and dagenham,1997,55974.250000
3,barking and dagenham,1998,60285.750000
4,barking and dagenham,1999,65320.833333
...,...,...,...
1165,yorks and the humber,2016,149015.583333
1166,yorks and the humber,2017,154819.916667
1167,yorks and the humber,2018,159936.416667
1168,yorks and the humber,2019,163584.583333


In [16]:
yearly_data = pd.merge(yearly_data, mean_from_monthly, on=['area', 'year'], how='inner')

In [17]:
yearly_data = yearly_data.drop(['date'], axis=1)

In [18]:
yearly_data

,area,median_salary,population_size,year,average_price
0,city of london,33020.0,6581.0,1999,171300.083333
1,barking and dagenham,21480.0,162444.0,1999,65320.833333
2,barnet,19568.0,313469.0,1999,136004.416667
3,bexley,18621.0,217458.0,1999,86777.666667
4,brent,18532.0,260317.0,1999,112157.416667
...,...,...,...,...,...
836,south east,30849.0,9133625.0,2018,323412.666667
837,south west,27956.0,5599735.0,2018,254385.500000
838,inner london,42153.0,3600203.0,2018,577009.083333
839,outer london,31031.0,5307878.0,2018,428121.666667


In [19]:
yearly_data.describe()

,median_salary,population_size,year,average_price
count,841.000000,8.410000e+02,841.000000,8.410000e+02
mean,28530.839477,2.553804e+06,2008.557669,2.957427e+05
std,6409.355450,8.092874e+06,5.732266,1.823522e+05
min,15684.000000,6.581000e+03,1999.000000,4.699950e+04
25%,24357.000000,2.147250e+05,2004.000000,1.802373e+05
50%,28034.000000,2.683350e+05,2009.000000,2.501425e+05
75%,31307.000000,3.493080e+05,2014.000000,3.625756e+05
max,61203.000000,5.597718e+07,2018.000000,1.363880e+06


### 5) Encode and scale features

In [20]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [21]:
encoder = LabelEncoder()

In [22]:
yearly_data['area'] = encoder.fit_transform(yearly_data['area'])

In [23]:
yearly_data

,area,median_salary,population_size,year,average_price
0,6,33020.0,6581.0,1999,171300.083333
1,0,21480.0,162444.0,1999,65320.833333
2,1,19568.0,313469.0,1999,136004.416667
3,2,18621.0,217458.0,1999,86777.666667
4,3,18532.0,260317.0,1999,112157.416667
...,...,...,...,...,...
836,34,30849.0,9133625.0,2018,323412.666667
837,35,27956.0,5599735.0,2018,254385.500000
838,20,42153.0,3600203.0,2018,577009.083333
839,31,31031.0,5307878.0,2018,428121.666667


In [24]:
y = yearly_data['average_price']
X = yearly_data.drop(['average_price'], axis=1)

In [25]:
scaler = StandardScaler()

In [26]:
X = scaler.fit_transform(X)

## Split to training and test

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

## Create the Model

In [29]:
from math import sqrt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [30]:
model = KNeighborsRegressor(n_neighbors=3)

In [31]:
model.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)

In [32]:
model.score(X_train, y_train)

0.8490415068117121

In [33]:
train_predictions = model.predict(X_train)

In [34]:
sqrt(mean_squared_error(y_train, train_predictions))

70691.18291708567

In [35]:
model.score(X_test, y_test)

0.7029556682340569

In [48]:
encoder.inverse_transform(yearly_data['area'])

array(['city of london', 'barking and dagenham', 'barnet', 'bexley',
       'brent', 'bromley', 'camden', 'croydon', 'ealing', 'enfield',
       'greenwich', 'hackney', 'hammersmith and fulham', 'haringey',
       'harrow', 'havering', 'hillingdon', 'hounslow', 'islington',
       'kensington and chelsea', 'kingston upon thames', 'lambeth',
       'lewisham', 'merton', 'newham', 'redbridge',
       'richmond upon thames', 'southwark', 'sutton', 'tower hamlets',
       'waltham forest', 'wandsworth', 'westminster', 'north east',
       'north west', 'east midlands', 'west midlands', 'london',
       'south east', 'south west', 'england', 'city of london',
       'barking and dagenham', 'barnet', 'bexley', 'brent', 'bromley',
       'camden', 'croydon', 'ealing', 'enfield', 'greenwich', 'hackney',
       'hammersmith and fulham', 'haringey', 'harrow', 'havering',
       'hillingdon', 'hounslow', 'islington', 'kensington and chelsea',
       'kingston upon thames', 'lambeth', 'lewisham', 